In [61]:
import cobra
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

expression_data = pd.read_csv('e_coli_core_expression.csv', index_col=0)
expression_data.sort_values(by=' reaction activity [mmol/gDW/h] ', ascending=False, inplace=True)
# expression_data

In [62]:
model = cobra.io.load_json_model('e_coli_core.json')
# model

Observations:
1. There are 95 reactions but only 73 have data!

# Excercise 1

No code required so answer is in the README.md

# Excercise 2

In [63]:
reactionsWithData = set(expression_data.index) # List of reactions with expression data from csv
reactionsInModel = set([rxn.id for rxn in model.reactions]) # List of reactions in the model
reactionsWithoutData = reactionsInModel - reactionsWithData # Set difference to find reactions without data
print(f"Number of reactions in the model: {len(reactionsInModel)}")
print(f"Number of reactions with expression data: {len(reactionsWithData)}")
print(f"Number of reactions without expression data: {len(reactionsWithoutData)}")
print(f"Reactions with expression data: {reactionsWithData}")
print(f"Reactions in the model but not in expression data: {reactionsWithoutData}")

Number of reactions in the model: 95
Number of reactions with expression data: 73
Number of reactions without expression data: 22
Reactions with expression data: {'ME1', 'AKGDH', 'ALCD2x', 'GLNS', 'PFK', 'MALS', 'G6PDH2r', 'THD2', 'CO2t', 'ACONTb', 'GND', 'ENO', 'GLUSy', 'ACALDt', 'FORt2', 'RPE', 'CYTBD', 'ADK1', 'PPC', 'NADTRHD', 'GLUDy', 'ACONTa', 'FUM', 'GLNabc', 'CS', 'PFL', 'PGI', 'TALA', 'ACKr', 'D_LACt2', 'PPS', 'FORt', 'FRUpts2', 'H2Ot', 'TKT1', 'ETOHt2r', 'GLUt2r', 'FBP', 'ATPS4r', 'GLCpts', 'RPI', 'PIt2r', 'NH4t', 'PYRt2', 'MDH', 'SUCCt2_2', 'PGM', 'SUCDi', 'LDH_D', 'TPI', 'O2t', 'PTAr', 'ICDHyr', 'GAPD', 'FRD7', 'SUCCt3', 'PGK', 'PGL', 'PDH', 'FBA', 'ACt2r', 'ME2', 'PPCK', 'PYK', 'GLUN', 'AKGt2r', 'FUMt2_2', 'MALt2_2', 'ACALD', 'SUCOAS', 'ICL', 'TKT2', 'NADH16'}
Reactions in the model but not in expression data: {'EX_succ_e', 'EX_h_e', 'EX_glc__D_e', 'ATPM', 'EX_akg_e', 'EX_pyr_e', 'EX_co2_e', 'EX_glu__L_e', 'EX_pi_e', 'EX_fum_e', 'EX_fru_e', 'EX_ac_e', 'EX_o2_e', 'EX_acald_

In [65]:
# Create a dictionary to map reaction IDs to their expression values
expression_dict = expression_data[' reaction activity [mmol/gDW/h] '].to_dict()

# reversible & irreversible reactions among the reactions with data
reversible_reactions = [rxn for rxn in model.reactions if rxn.id in expression_dict and rxn.reversibility]
irreversible_reactions = [rxn for rxn in model.reactions if rxn.id in expression_dict and not rxn.reversibility]
len(reversible_reactions), len(irreversible_reactions)

# NOTICE THAT GLUCOSE EXCHANGE & ATPM ARE NOT IN THE EXPRESSION DATA!!! Also the lengths of lists match the expected values :D

(39, 34)

In [67]:
# 1. For reversible reactions with expression data, set lower and upper bounds to -value and +value usinf expression_dict!
for rxn in reversible_reactions:
    value = expression_dict[rxn.id]
    rxn.lower_bound = -value
    rxn.upper_bound = value
    

# 2. For irreversible reactions with expression data, set lower and upper bounds to 0 and value or -value and 0 depending on the original bounds
# (THIS IS TRICKY BECAUSE THE DIRECTIONS ARE NOT CONSISTENT)
for rxn in irreversible_reactions:
    value = expression_dict[rxn.id]
    if rxn.upper_bound > 0: 
        rxn.upper_bound = value
    elif rxn.lower_bound < 0: 
        rxn.lower_bound = -value
        
# 3. For reactions without data, leave bounds as is. No change required since its in the NO DATA REACTIONS list

# 4. For glucose exchange reaction, set bounds to -1000 and 1000
glucose_exchange = model.reactions.get_by_id('EX_glc__D_e')
glucose_exchange.lower_bound = -1000
glucose_exchange.upper_bound = 1000

# 5. For ATPM reaction, leave bounds as is. No change required since its in the NO DATA REACTIONS list

In [73]:
# print a table listing each reaction’s lower and upper flux bound after implementing the above
for rxn in model.reactions:
    print(f"{rxn.id}: Lower Bound = {rxn.lower_bound}, Upper Bound = {rxn.upper_bound}")

PFK: Lower Bound = 0.0, Upper Bound = 12.12
PFL: Lower Bound = 0.0, Upper Bound = 1.0
PGI: Lower Bound = -13.12, Upper Bound = 13.12
PGK: Lower Bound = -23.13, Upper Bound = 23.13
PGL: Lower Bound = 0.0, Upper Bound = 8.12
ACALD: Lower Bound = -1.16, Upper Bound = 1.16
AKGt2r: Lower Bound = -3.1, Upper Bound = 3.1
PGM: Lower Bound = -20.01, Upper Bound = 20.01
PIt2r: Lower Bound = -6.03, Upper Bound = 6.03
ALCD2x: Lower Bound = -9.01, Upper Bound = 9.01
ACALDt: Lower Bound = -2.29, Upper Bound = 2.29
ACKr: Lower Bound = -1.19, Upper Bound = 1.19
PPC: Lower Bound = 0.0, Upper Bound = 2.56
ACONTa: Lower Bound = -25.35, Upper Bound = 25.35
ACONTb: Lower Bound = -25.35, Upper Bound = 25.35
ATPM: Lower Bound = 8.39, Upper Bound = 1000.0
PPCK: Lower Bound = 0.0, Upper Bound = 25.23
ACt2r: Lower Bound = -3.23, Upper Bound = 3.23
PPS: Lower Bound = 0.0, Upper Bound = 2.5
ADK1: Lower Bound = -30.57, Upper Bound = 30.57
AKGDH: Lower Bound = 0.0, Upper Bound = 24.35
ATPS4r: Lower Bound = -60.5, U

# Excercise 3

# Excercise 4